<a href="https://colab.research.google.com/github/ortenburger/apoHackathon/blob/master/Fuzzy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!pip install fuzzywuzzy[speedup]
import os, json
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir("gdrive/My Drive/Hackathon")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


FileNotFoundError: ignored

In [9]:
from fuzzywuzzy import fuzz
fuzz.ratio('Karl-Ferdinand-Broll-Str. (D)', 'Karl-Ferdinand-Broll-Straße')

86

In [0]:

with open("locations.json") as json_file:  
    data = json.load(json_file)

In [71]:
print(data[100])
comparedata = []
for date in data:
  address = date['address'].split(',')
  try:
    ort = address[ -2].split(' ', 2)[2].strip()
  except:
    print(address)
    ort = None
    address = [None,None,None]
  comparedata.append({'IK': date['IK'], 'Ort': ort, 'Strasse': address[0].strip()})
print(comparedata[100])

{'IK': '260102058', 'year': '2016', 'standort': '00', 'address': 'Röpersberg 47 , 23909 Ratzeburg , Germany'}
{'IK': '260102058', 'Ort': 'Ratzeburg', 'Strasse': 'Röpersberg 47'}


In [0]:
df = pd.read_excel('Bilanzdaten_Krankenhäuser.xlsx')
df['IK'] = None

In [81]:
!ls

 2013_Qualitätsberichte.zip	 adresses2019.txt
 2013_Qualitätsberichte.zip.h5	 Bilanzdaten.h5
 2014_Qualitätsberichte.zip	 Bilanzdaten_Krankenhäuser.xlsx
 2015_Qualitätsberichte.zip	 locations.json
 2016_Qualitätsberichte.zip	'Qualitätsberichte (1).h5'
 2017_Qualitätsberichte.zip	 Qualitätsberichte.h5
 adresses2019.json


In [85]:
from fuzzywuzzy import fuzz
for index, row in df.iterrows():
  #print(row['Ort'], row['GKZ Straße'])
  for line in comparedata:
    if fuzz.ratio(str(row['Ort']), str(line['Ort'])) > 90:
      if fuzz.ratio(str(row['GKZ Straße']), str(line['Strasse'])) > 60:
        df.loc[index,'IK'] = line['IK']
        break
df.head()

,Unnamed: 0,Name des Unternehmens,Ort,Letzter Bilanzstichtag,Umsatzerlöse tsd EUR Letztes verf. Jahr,COSMOS Geocode (KGS-Version),KGS Geocode,KGS Ort / Gemeinde,GKZ PLZ8,GKZ Straße,Jahresüberschuss / Jahresfehlbetrag tsd EUR Letztes verf. Jahr,Jahresüberschuss / Jahresfehlbetrag tsd EUR 2017,Jahresüberschuss / Jahresfehlbetrag tsd EUR 2016,Jahresüberschuss / Jahresfehlbetrag tsd EUR 2015,Cashflow tsd EUR Letztes verf. Jahr,Cashflow tsd EUR 2017,Cashflow tsd EUR 2016,Cashflow tsd EUR 2015,Bilanzsumme tsd EUR Letztes verf. Jahr,Bilanzsumme tsd EUR 2017,Bilanzsumme tsd EUR 2016,Bilanzsumme tsd EUR 2015,Eigenkapital tsd EUR Letztes verf. Jahr,Eigenkapital tsd EUR 2017,Eigenkapital tsd EUR 2016,Eigenkapital tsd EUR 2015,Eigenkapital tsd EUR Letztes verf. Jahr.1,Eigenkapital tsd EUR 2019,Eigenkapital tsd EUR 2018,Eigenkapital tsd EUR 2017.1,Eigenkapital tsd EUR 2016.1,Eigenkapital tsd EUR 2015.1,Personalaufwand (GKV) tsd EUR Letztes verf. Jahr,Personalaufwand (GKV) tsd EUR 2017,Personalaufwand (GKV) tsd EUR 2016,Personalaufwand (GKV) tsd EUR 2015,Personalaufwand (GKV) tsd EUR 2018,Materialaufwand (GKV) tsd EUR Letztes verf. Jahr,Materialaufwand (GKV) tsd EUR 2017,Materialaufwand (GKV) tsd EUR 2016,Materialaufwand (GKV) tsd EUR 2015,Materialaufwand (GKV) tsd EUR 2018,IK
0,1.0,Asklepios Kliniken Hamburg GmbH,Hamburg,2017.0,1.13457e+06,2019-1 (02/2019),NaN,NaN,NaN,Aalkrautweg (D),75301,75301,77181.4,60564.2,151433,151433,152015,134133,1.74903e+06,1.74903e+06,1.68394e+06,1.59043e+06,757850,757850,682549,605368,757850,n.v.,n.v.,757850,682549,605368,599226,599226,608138,586050,n.v.,324484,324484,330499,304958,n.v.,None
1,2.0,Gesundheit Nord gGmbH Klinikverbund Bremen,Bremen,2017.0,630699,2019-1 (02/2019),NaN,NaN,28211-010,Kurfürstenallee 104 - 130 (G),-21278.7,-21278.7,-11451.6,-16375.9,7045.91,7045.91,15918.4,10468.3,916279,916279,904767,945714,183514,183514,196792,203244,183514,n.v.,n.v.,183514,196792,203244,416741,416741,395558,378344,n.v.,176981,176981,173509,173457,n.v.,None
2,3.0,Städtisches Klinikum München GmbH,München,2017.0,630091,2019-1 (02/2019),NaN,NaN,80337-020,Thalkirchner Str. 48 - 92 (G),23194.7,23194.7,1512.93,-30656.7,56407.3,56407.3,32599.2,1091.8,658674,658674,673855,603954,76406.3,76406.3,59189.6,43089.6,76406.3,n.v.,n.v.,76406.3,59189.6,43089.6,419988,419988,415815,427834,n.v.,174343,174343,168954,172262,n.v.,None
3,4.0,Marienhaus Kliniken GmbH,Waldbreitbach,2017.0,568267,2019-1 (02/2019),NaN,NaN,56588-010,Margaretha-Flesch-Str. (D),-11556.1,-11556.1,8548.24,7656.55,2666.52,2666.52,37922.8,32483.1,337072,337072,464694,499745,134785,134785,149207,174561,134785,n.v.,n.v.,134785,149207,174561,391782,391782,372436,360765,n.v.,119160,119160,122552,122602,n.v.,260711658
4,5.0,Klinikum Region Hannover GmbH,Hannover,2016.0,519439,2019-1 (02/2019),NaN,NaN,30459-005,Stadionbrücke 6 - 6 (G),2627.81,n.v.,2627.81,-9032.79,43002,n.v.,43002,24438.7,626359,n.v.,626359,642588,50912.2,n.v.,50912.2,48570,50912.2,n.v.,n.v.,n.v.,50912.2,48570,328473,n.v.,328473,327533,n.v.,129537,n.v.,129537,117832,n.v.,260321781


In [0]:

#df['Umsatzerlöse\ntsd EUR\nLetztes verf. Jahr'] = df['Umsatzerlöse\ntsd EUR\nLetztes verf. Jahr'].astype(float)
df = df.loc[:,'Umsatzerlöse\ntsd EUR\nLetztes verf. Jahr'].apply(pd.to_numeric, errors='coerce')

In [93]:
store = pd.HDFStore('Bilanzdaten.h5')
store.append('Bilanzen', df)
store.close()

TypeError: ignored